In [6]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['ALK']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('anaplastic lymphoma kinase', 1253),
 ('activin receptor like kinase', 24),
 ('anaplastic lymphoma kinase gene', 17),
 ('anaplastic lymphoma receptor tyrosine kinase', 16),
 ('alkaline phosphatase', 7),
 ('activin like kinase', 6),
 ('anaplastic lymphomakinase', 5),
 ('anaplastic lymphoma tyrosine kinase', 4),
 ('alk', 3),
 ('alkannin', 3),
 ('anaplastic lymphoma receptor tyrosine kinase gene', 3),
 ('anaplastic large cell lymphoma kinase', 3),
 ('alk receptor tyrosine kinase gene', 3)]

In [ ]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-17 01:53:47] indra.ontology.bio.ontology - Initializing INDRA bio ontology for the first time, this may take a few minutes...
INFO: [2020-10-17 01:53:47] indra.ontology.bio.ontology - Adding nodes...
INFO: [2020-10-17 01:53:56] indra.ontology.bio.ontology - Adding xrefs...
INFO: [2020-10-17 01:54:08] indra.ontology.bio.ontology - Adding hierarchy...
INFO: [2020-10-17 01:54:25] indra.ontology.bio.ontology - Building name lookup...
INFO: [2020-10-17 01:54:31] indra.ontology.bio.ontology - Finished initializing bio ontology...
INFO: [2020-10-17 01:54:31] indra.ontology.bio.ontology - Caching INDRA bio ontology at /home/ubuntu/.indra/bio_ontology/1.3/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [7]:
d = load_disambiguator('ALK')

In [9]:
result = [d.grounding_dict['ALK'], d.names, d.pos_labels]

In [10]:
result

[{'2p23': 'HGNC:427',
  'activin receptor like kinase': 'FPLX:RSTK1',
  'alkaline phosphatase': 'MESH:D000469',
  'anaplastic lymphoma kinase': 'HGNC:427',
  'anaplastic lymphomakinase': 'HGNC:427'},
 {'HGNC:427': 'ALK',
  'MESH:D000469': 'Alkaline Phosphatase',
  'FPLX:RSTK1': 'RSTK1'},
 ['FPLX:RSTK1', 'HGNC:427', 'MESH:D000469']]

In [11]:
grounding_map, names, pos_labels = [{'2p23': 'HGNC:427',
  'activin receptor like kinase': 'FPLX:RSTK1',
  'alkaline phosphatase': 'MESH:D000469',
  'anaplastic lymphoma kinase': 'HGNC:427',
  'anaplastic lymphomakinase': 'HGNC:427'},
 {'HGNC:427': 'ALK',
  'MESH:D000469': 'Alkaline Phosphatase',
  'FPLX:RSTK1': 'RSTK1'},
 ['FPLX:RSTK1', 'HGNC:427', 'MESH:D000469']]

In [12]:
excluded_longforms = []

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [15]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [16]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [17]:
intersection1

[('HGNC:17981', 'HGNC:17981', 29)]

In [18]:
intersection2

[('ungrounded', 'HGNC:17981', 0)]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-27 03:04:32] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-27 03:04:46] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9778344614234659 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [18]:
classifier.stats

{'label_distribution': {'HGNC:427': 881, 'FPLX:RSTK1': 19, 'MESH:D000469': 6},
 'f1': {'mean': 0.977834, 'std': 0.01057},
 'precision': {'mean': 0.973822, 'std': 0.014626},
 'recall': {'mean': 0.983456, 'std': 0.006941},
 'MESH:D000469': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'HGNC:427': {'f1': {'mean': 0.991568, 'std': 0.003509},
  'pr': {'mean': 0.998864, 'std': 0.002273},
  'rc': {'mean': 0.984412, 'std': 0.007227}},
 'FPLX:RSTK1': {'f1': {'mean': 0.616667, 'std': 0.331662},
  'pr': {'mean': 0.55, 'std': 0.331662},
  'rc': {'mean': 0.75, 'std': 0.387298}}}

In [19]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [20]:
disamb.dump(model_name, results_path)

In [21]:
print(disamb.info())

Disambiguation model for ALK

Produces the disambiguations:
	ALK*	HGNC:427
	Alkaline Phosphatase*	MESH:D000469
	RSTK1*	FPLX:RSTK1

Class level metrics:
--------------------
Grounding           	Count	F1     
                 ALK*	881	0.99157
               RSTK1*	 19	0.61667
Alkaline Phosphatase*	  6	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.97783
	Precision:	0.97382
	Recall:		0.98346

* Positive labels
See Docstring for explanation



In [22]:
model_to_s3(disamb)